In [19]:
import jax.experimental.ode as ode
import jax
import ticktack
import matplotlib.pyplot as pyplot

In [20]:
parameters = (774.86, 0.25, 0.8, 6.44)
model = ticktack.load_presaved_model("Guttler15", production_rate_units="atoms/cm^2/s")
model.compile()

STEADY_PROD = model.equilibrate(target_C_14=707.)
STEADY_STATE = model.equilibrate(production_rate=STEADY_PROD)
PROD_COEFFS = model._production_coefficients
MATRIX = model._matrix
GROWTH = model

del model

In [21]:
@jax.tree_util.Partial
@jax.jit
def production(t, args):
    start_time, duration, phase, area = jax.numpy.array(args)
    middle = start_time + duration / 2.
    height = area / duration

    gauss = height * jax.numpy.exp(- ((t - middle) / (1. / 1.93516 * duration)) ** 16.)
    sine = 1.8803862513018528 + 0.18 * 1.8803862513018528 *\
        jax.numpy.sin(2 * jax.numpy.pi / 11 * t + phase * 2 * jax.numpy.pi / 11) 
        
    return (sine + gauss) * 3.747273140033743

In [22]:
@jax.tree_util.Partial
@jax.jit
def derivative(y, t, args, /, matrix=MATRIX, production=production, prod_coeffs=PROD_COEFFS): 
    
    ans = jax.numpy.matmul(matrix, y) 
    production_rate_constant = production(t, args)
    production_term = prod_coeffs * production_rate_constant 
    return ans + production_term 

In [23]:
@jax.tree_util.Partial
@jax.jit
def solve(y_initial, time, args, /, dydx=derivative):
    states = ode.odeint(dydx, y_initial, time, args)
    return states[:, 1]

In [24]:
with open("miyake12.csv") as data:
    _ = next(data)  # String titles 
    data = jax.numpy.array([row.strip().split(" ") for row in data],\
        dtype=jax.numpy.float64)
    data = data.T

In [48]:
@jax.tree_util.Partial(jax.jit, static_argnums=(3))
def bin_data(data, time_out, growth, oversample):
    masked = jax.numpy.linspace(0, 1, oversample)
    # So the kernel represents the fraction of decimals between 0 and 1 that are less than the length of the growth season normalised
    kernel = (masked < jax.numpy.count_nonzero(growth)/12)
    
    # These top two return the index of the first one and the first zero
    first1 = jax.numpy.where(growth == 1, size=1)[0][0]
    first0 = jax.numpy.where(growth == 0, size=1)[0][0]

    # Here we zone onto the start of wrapped seasons
    all1s = jax.numpy.where(growth == 1, size=12)[0]
    after1 = jax.numpy.where(all1s > first0, all1s, 0)
    after1 = after1.at[jax.numpy.nonzero(after1, size=1)].get()[0]

    # difference between the wrapped and non-wrapped 
    num = jax.lax.sub(first1, after1)

    # if the season is not wrapped then return the index of the season start.
    # if the seasoon is wrapped return the start and the difference
    val = jax.lax.cond(num == 0, lambda: first1, lambda: after1)
    # Make sure that the tree doesn't grow all year around 
    shifted_index = jax.lax.cond(jax.numpy.all(growth == 1), lambda: 0, lambda: val)

    def fun(i, val, /, oversample=oversample, data=data, kernel=kernel, shifted_index=shifted_index):
        # So val is the array that is getting updated here
        translated_index = i * oversample + shifted_index * oversample // 12
        year_from_index = jax.lax.dynamic_slice(data, (translated_index,), (oversample,))
        selected_data = jax.numpy.multiply(year_from_index, kernel)
        year_mean = jax.numpy.array([jax.numpy.sum(selected_data) / (jax.numpy.sum(kernel))])

        # So this will update val 
        solution = jax.lax.dynamic_update_slice(val, year_mean, (i,))
        return solution 

    binned_data = jax.numpy.zeros((len(time_out),))
    binned_data = jax.lax.fori_loop(0, len(time_out), fun, binned_data)

    return binned_data

In [26]:
oversample = 1008
growth = jax.numpy.array([0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0], dtype=jax.numpy.float64)
time_out = jax.numpy.linspace(data[0].min(), data[0].max() + 2, (data[0].size + 1) * oversample)

In [27]:
@jax.tree_util.Partial
@jax.jit
def log_likelihood_binned(args, /, data=data, func=solve, bin=bin_data, y0=STEADY_STATE,\
    growth=growth, time_out=time_out, oversample=oversample):

    solution = func(y0, time_out, args)
    solution = bin(solution, data[0], growth, oversample)
    solution = (solution - STEADY_STATE[1]) / STEADY_STATE[1]
    solution += jax.numpy.mean(data[1][:4])
    chi_squared = (solution - data[1]) ** 2 / data[2] ** 2
    return - 0.5 * jax.numpy.sum(chi_squared)

In [28]:
@jax.tree_util.Partial
@jax.jit
def log_likelihood(args, /, data=data, func=solve, y0=STEADY_STATE):

    solution = func(y0, data[0], args)
    solution = (solution - STEADY_STATE[1]) / STEADY_STATE[1]
    solution += jax.numpy.mean(data[1][:4])
    chi_squared = (solution - data[1]) ** 2 / data[2] ** 2
    return - 0.5 * jax.numpy.sum(chi_squared)

In [29]:
%%timeit
log_likelihood(parameters)

The slowest run took 6.94 times longer than the fastest. This could mean that an intermediate result is being cached.
377 µs ± 331 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%%timeit
log_likelihood_binned(parameters)

2.01 ms ± 486 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
jac = jax.jit(jax.grad(log_likelihood))
jac_binned = jax.jit(jax.grad(log_likelihood_binned))

In [32]:
%%timeit
jac(parameters)

1.49 ms ± 350 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%%timeit
jac_binned(parameters)

119 ms ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
@jax.jit
def sin_likelihood(time_out):
    model = jax.numpy.sin(time_out)
    data = jax.random.normal(jax.random.PRNGKey(0), (time_out.shape))
    error = jax.random.normal(jax.random.PRNGKey(0), (time_out.shape))
    likelihood = (model - data) ** 2 / error ** 2
    return - 0.5 * likelihood

@jax.jit
def binned_sin_likelihood(time_out, /, bins=data[0], growth=growth, oversample=oversample):
    model = jax.numpy.sin(time_out)
    model = bin_data(model, bins, growth, oversample)
    data = jax.random.normal(jax.random.PRNGKey(0), (model.shape))
    error = jax.random.normal(jax.random.PRNGKey(0), (model.shape))
    likelihood = (model - data) ** 2 / error ** 2
    return - 0.5 * likelihood

In [89]:
cos = jax.jit(jax.jacobian(sin_likelihood))
binned_cos = jax.jit(jax.jacobian(binned_sin_likelihood))

In [65]:
%%timeit
sin_likelihood(time_out)

817 µs ± 48.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
%%timeit
binned_sin_likelihood(time_out)

465 µs ± 148 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [90]:
%%timeit
cos(time_out)

In [ ]:
%%timeit
binned_cos(time_out)

(DeviceArray([ 0,  0,  0, ..., 27, 27, 27], dtype=int64),
 DeviceArray([  252,   253,   254, ..., 27969, 27970, 27971], dtype=int64))